> ## **Facial Detection and Recognition Project** ##

We will start by importing the libraries we will need for this project.

In [1]:
import cv2
import gradio as gr
import numpy as np
from PIL import Image
from keras.models import load_model

We are going to use pre-trained models as train a facial detection model is time consuming, requires a lot of data and a lot of computing power. We will use the pre-trained models from the OpenCV library. We will use the Haar Cascade Classifier for the face detection. 

In [2]:
# Load the pre-trained Haar Cascade Classifier for face detection
face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [3]:
# Load the pre-trained classification models
emotion_model = load_model('emotionModel.hdf5')
gender_model = load_model('genderModel.hdf5')


We will define the stored emotions and genders so that they can be easily understood by the user/reader.

In [4]:
# Mapping of class indices to human-readable labels
emotion_labels = ["Angry","Disgust","Fear", "Happy", "Surprise", "Neutral"]
gender_labels = ["Male", "Female"]

Now let us define a function that will detect the face in the image and identify the associated emotions and gender of the person in question. We will use the Haar Cascade Classifier for the face detection. The function also resizes the input images based upon the requirements of the model.


In [5]:
def detect_and_classify_faces(input_pil_image):

    if isinstance(input_pil_image, Image.Image):
        input_array = np.array(input_pil_image)
    else:
        input_array = input_pil_image


    if input_array.shape[2] == 4:
        input_array = cv2.cvtColor(input_array, cv2.COLOR_RGBA2RGB)

    gray = cv2.cvtColor(input_array, cv2.COLOR_RGB2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5)
    
    print("Number of detected faces:", len(faces))


    output_array = input_array.copy()


    for (x, y, w, h) in faces:
        face = input_array[y:y + h, x:x + w]

         # Resize face for gender classification
        resized_face_gender = cv2.resize(face, (150, 150))
        gender_idx = np.argmax(gender_model.predict(np.expand_dims(resized_face_gender, axis=0)))
        predicted_gender = gender_labels[gender_idx]

        # Resize face and convert to grayscale for emotion classification
        resized_face_emotion = cv2.resize(face, (64, 64))
        gray_face_emotion = cv2.cvtColor(resized_face_emotion, cv2.COLOR_RGB2GRAY)
        gray_face_emotion = gray_face_emotion.reshape(64, 64, 1)
        emotion_idx = np.argmax(emotion_model.predict(np.expand_dims(gray_face_emotion, axis=0)))
        predicted_emotion = emotion_labels[emotion_idx]

        cv2.rectangle(output_array, (x, y), (x + w, y + h), (255, 0, 0), 2)
        cv2.putText(output_array, f"Gender: {predicted_gender}", (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        cv2.putText(output_array, f"Emotion: {predicted_emotion}", (x, y - 30), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    return Image.fromarray(output_array)


Now, that we have created the model. It is only reasonable to use a GUI so that we might make it user-friendly and accesible. For this, I have used the gradios library.

In [6]:
iface = gr.Interface(
    fn=detect_and_classify_faces,
    inputs=gr.inputs.Image(type="pil", label="Upload an image or video file"),
    outputs=gr.outputs.Image(type="pil")
)

C:\Users\Azlaan\AppData\Local\Temp\ipykernel_11248\991791201.py:3: GradioDeprecationWarning: Usage of gradio.inputs is deprecated, and will not be supported in the future, please import your component from gradio.components
  inputs=gr.inputs.Image(type="pil", label="Upload an image or video file"),
C:\Users\Azlaan\AppData\Local\Temp\ipykernel_11248\991791201.py:3: GradioDeprecationWarning: `optional` parameter is deprecated, and it has no effect
  inputs=gr.inputs.Image(type="pil", label="Upload an image or video file"),
C:\Users\Azlaan\AppData\Local\Temp\ipykernel_11248\991791201.py:4: GradioDeprecationWarning: Usage of gradio.outputs is deprecated, and will not be supported in the future, please import your components from gradio.components
  outputs=gr.outputs.Image(type="pil")


In [7]:
iface.launch()

Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Number of detected faces: 1
1/1 [==============================] - 0s 223ms/step
